In [1]:
import json
import time
import torch
import transformers
import wandb
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from transformers import BitsAndBytesConfig


# nltk 다운로드 (최초 1회)
nltk.download('punkt')

# wandb 초기화
wandb.init(project="inference_evaluations", name="8B_4bit_quantization")

# corpus.json 파일 로드 (UTF-8 인코딩)
with open("./corpus.json", "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

# 10개만 추론
evaluation_data = evaluation_data[:20]

# 허깅페이스 방식으로 모델 및 파이프라인 초기화
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


generator = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    quantization_config=quantization_config  # 4비트 양자화를 적용합니다.
)
generator.model.eval()

# 시스템 프롬프트 정의
SYSTEM_PROMPT = (
    "You are a helpful AI assistant. Please answer the user's questions kindly. "
    "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."
)

# 채팅 템플릿을 적용하여 프롬프트를 생성하는 함수
def make_prompt(system_msg: str, user_msg: str) -> str:

    msg = f"""
다음은 유튜브 타임라인 요약 문서입니다. 이 내용을 분석하여 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.
중요하지 않더라도 알면 좋을 추가 핵심 용어도 더해주세요.

[타임라인 요약]:
{user_msg}

[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는 여러 개의 키포인트 리스트로 구성됩니다.
- 반드시 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

"""

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": msg}
    ]
    prompt = generator.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return prompt

# 종료 토큰 설정
terminators = [
    generator.tokenizer.eos_token_id,
    generator.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]



# 평가 루프: corpus.json에 있는 각 예제에 대해 추론 수행
for idx, example in enumerate(evaluation_data):
    print(f"\n==== Example {idx+1} ====")

    # 사용자 입력과 프롬프트 생성
    user_input = example["input"]
    prompt = make_prompt(SYSTEM_PROMPT, user_input)

    # 입력 토큰 인코딩
    encoded_input = generator.tokenizer(prompt, return_tensors="pt")
    encoded_input = {k: v.to(generator.model.device) for k, v in encoded_input.items()}
    prompt_length = encoded_input["input_ids"].shape[1]

    # 속도 및 메모리 측정을 위한 시작 시간 기록 및 GPU 메모리 초기화
    start_time = time.time()
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(generator.model.device)

    # 모델 생성: 토큰별 score 반환 포함
    outputs = generator.model.generate(
        **encoded_input,
        max_new_tokens=2048,
        eos_token_id=terminators[0],
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # GPU 동기화 후 종료 시간 기록
    if torch.cuda.is_available():
        torch.cuda.synchronize(generator.model.device)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # GPU 메모리 사용량 측정 (Peak 메모리, MB 단위)
    if torch.cuda.is_available():
        peak_memory = peak_memory = round(torch.cuda.max_memory_allocated(generator.model.device) / (1024 ** 3), 1)

    else:
        peak_memory = 0

    # 생성된 전체 토큰에서 프롬프트 이후의 candidate 토큰 추출
    generated_ids = outputs.sequences[0]
    candidate_ids = generated_ids[prompt_length:]
    candidate_text = generator.tokenizer.decode(candidate_ids, skip_special_tokens=True).strip()

    print("Candidate output:")
    print(candidate_text)
    print(f"Inference Time: {elapsed_time:.2f} seconds")
    print(f"Peak GPU Memory Usage: {peak_memory:.2f} GB")

    # 참조 텍스트: evaluation_data의 "output"을 JSON 문자열로 변환
    reference_text = json.dumps(example["output"], ensure_ascii=False, indent=2)

    # BLEU 점수 계산 (공백 기준 토큰화)
    candidate_tokens = candidate_text.split()
    reference_tokens = reference_text.split()
    bleu = sentence_bleu(
        [reference_tokens],
        candidate_tokens,
        smoothing_function=SmoothingFunction().method1
    )
    print(f"BLEU score: {bleu:.4f}")

    # 로그 확률 계산: 각 생성 토큰에 대해 log‑softmax 후 선택된 토큰의 로그 확률 합산
    log_score = 0.0
    for i, step_scores in enumerate(outputs.scores):
        log_probs = torch.log_softmax(step_scores, dim=-1)
        token_id = candidate_ids[i].unsqueeze(0)  # shape: (1,)
        token_log_prob = log_probs[0, token_id].item()
        log_score += token_log_prob
    print(f"Log probability score (sum): {log_score:.4f}")

    # wandb에 메트릭 로깅
    wandb.log({
        "example_index": idx,
        "BLEU_score": bleu,
        "Inference_time_sec": elapsed_time,
        "Peak_GPU_Memory_GB": peak_memory,
        "Log_probability_score": log_score
    })

wandb.finish()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: Currently logged in as: iamkimhongil92 (iamkimhongil92-lumenasoft) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



==== Example 1 ====


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Candidate output:
[키포인트]
[
  {
    "term": "캐릭터 해금 시스템",
    "description": "게임 내에서 플레이어의 성취를 인정하고 보상을 제공하기 위해 캐릭터를 해금하는 기능을 구현하는 시스템. 이를 통해 플레이어의 게임 지속성을 높이고, 다양한 업적 시스템을 활용할 수 있게 된다."
  },
  {
    "term": "버튼 추가",
    "description": "게임 내에서 새로운 기능을 수행하도록 하는 버튼을 추가하는 과정. 캐릭터 해금 시스템에서는 감자 농부와 콩 농부와 같은 새로운 캐릭터를 제어할 수 있게 하는 버튼을 추가한다."
  },
  {
    "term": "Ctrl+D",
    "description": "Windows 운영 체제에서 복사 기능을 수행하는 단축키. 캐릭터 해금 시스템에서는 버튼을 복사하여 다른 위치에 붙여 사용할 수 있게 한다."
  },
  {
    "term": "잠금 상태 버튼",
    "description": "기본 캐릭터는 활성화 상태로 두고, 나머지 캐릭터에 대해 잠금 상태를 설정하는 버튼. 이를 통해 플레이어는 기본 캐릭터를 계속 사용하면서도 다른 캐릭터를 해금할 수 있게 된다."
  },
  {
    "term": "업적 달성",
    "description": "게임 내에서 특정 조건을 충족하면 플레이어에게 보상을 주는 시스템. 캐릭터 해금 시스템에서는 특정 캐릭터를 언락하는 데 필요한 조건을 설정하고, 이를 달성하면 버튼에 변화가 생긴다."
  },
  {
    "term": "스크립트",
    "description": "프로그래밍 언어를 사용하여 게임의 로직을 구현하는 코드. 캐릭터 해금 시스템에서는 언락 조건을 정의하고, 조건 달성 시 UI에 변화를 주기 위한 로직을 스크립트로 구현한다."
  },
  {
    "term": "플레이어 선호도",
    "description": "플레이어가 선호하는 게임 설정이나 캐릭터 해금 상태

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Candidate output:
```json
[
  {
    "term": "유니티",
    "description": "유니티는 Unity Engine로, 2D 및 3D 게임 개발을 위한 다목적 게임 엔진입니다."
  },
  {
    "term": "오디오 시스템",
    "description": "게임 내에서 오디오 콘텐츠를 효과적으로 관리하고 재생하는 데 사용되는 시스템입니다. 배경음악, 효과음 등을 포함합니다."
  },
  {
    "term": "오디오 매니저",
    "description": "유니티에서 오디오 콘텐츠를 관리하는 오브젝트입니다. 오디오 소스를 생성하고, 재생하고, 제어하는 역할을 합니다."
  },
  {
    "term": "오디오 소스",
    "description": "오디오 매니저에서 사용되는 오디오 파일이나 오디오 효과를 의미합니다. 배경음악, 효과음 등이 포함됩니다."
  },
  {
    "term": "오디오 클립",
    "description": "오디오 파일이 아닌 오디오 매니저에서 오디오 소스를 관리하는 단위입니다. 음악 파일 대신 오디오 클립을 사용하여 오디오를 재생합니다."
  },
  {
    "term": "채널",
    "description": "오디오 소스를 여러 개의 경로로 나누어 관리하는 방법입니다. 여러 개의 효과음을 동시에 재생할 수 있게 합니다."
  },
  {
    "term": "배경음악",
    "description": "게임 내 배경에 계속해서 재생되는 음악입니다. 게임의 분위기를 설정하는 데 중요한 역할을 합니다."
  },
  {
    "term": "효과음",
    "description": "게임 내에서 특정 이벤트나 행동에 반응하여 재생되는 소리가 있는 소리를 의미합니다. 예를 들어, 아이템을 획득할 때 나오는 효과음 등이 있습니다."
  },
  {
    "term": "사운드 품질",
    "description": "오디오 

BLEU_score,▁▇▂▃▂▅█▁▅▁▅▂▄▁▁▁▂▇▇▆
Inference_time_sec,▆▄▅▅▃▂▁▃▂█▅▅▆▃▄▄▁▂▂▃
Log_probability_score,▃▅▃▆█▇▇▇▇▁▃▅▄▄▅▇█▇▆▇
Peak_GPU_Memory_GB,▁▆▅▆▅▃▁▂▃▆█▆▆▅▅▅▂▁▁▂
example_index,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
BLEU_score,0.11689
Inference_time_sec,17.37664
Log_probability_score,-93.46854
Peak_GPU_Memory_GB,15.9
example_index,19
